# 1.데이터 불러오기
## 2. 필요한 라이브러리 설치
mask와 nomask 폴더를 각각 생성하여 
mask 폴더에 with_mask
nomask 폴더에 without_mask 데이터를 저장

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input


mask = 'C:/Users/gram/Desktop/파이썬메시업프로젝트/Face-Mask-Detection-master/Face-Mask-Detection-master/dataset/with_mask/'
nomask = 'C:/Users/gram/Desktop/파이썬메시업프로젝트/Face-Mask-Detection-master/Face-Mask-Detection-master/dataset/without_mask/'

# 파일 가져오기

In [2]:
file_list1 = os.listdir(mask) # path에 존재하는 파일 목록 가져오기
file_list2 = os.listdir(nomask)
 
file_list1_num = len(file_list1)
file_list2_num = len(file_list2)
 
file_num = file_list1_num + file_list2_num

In [3]:
num = 0;
all_img = np.float32(np.zeros((file_num, 224, 224, 3))) 
all_label = np.float64(np.zeros((file_num, 1)))
 
for img_name in file_list1:
    img_path = mask+img_name
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 0 # nomask
    num = num + 1

In [4]:
for img_name in file_list2:
    img_path = nomask+img_name
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 1 # mask
    num = num + 1

# 데이터 훈련시키기
train 데이터와 test 데이터를 8 대 2의 비율로 적절하게 훈련시킴

In [5]:
# 데이터셋 섞기(적절하게 훈련되게 하기 위함) 
n_elem = all_label.shape[0]
indices = np.random.choice(n_elem, size=n_elem, replace=False)
 
all_label = all_label[indices]
all_img = all_img[indices]

In [6]:
# 훈련셋 테스트셋 분할
num_train = int(np.round(all_label.shape[0]*0.8))
num_test = int(np.round(all_label.shape[0]*0.2))
 
train_img = all_img[0:num_train, :, :, :]
test_img = all_img[num_train:, :, :, :] 
 
train_label = all_label[0:num_train]
test_label = all_label[num_train:]

# 전이학습 모델 사용
MobileNetV2를 사용
# 레이어층 구성
Dense 레이어 입력층을 256개 출력층을 1개로 해서 사용 (sigmoid 함수 사용)
# 하이퍼 파라미터 튜닝
lr 값을 0.001로 설정 optimizers은 Adam으로 설정
epochs은 10번 지정 batch_size는 16으로 지정

In [7]:
# create the base pre-trained model
IMG_SHAPE = (224, 224, 3)

base_model = MobileNetV2(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
base_model.trainable = False  #값을 고정 시켜줌, 이 레이어는 학습 안시킴
base_model.summary()
print("Number of layers in the base model: ", len(base_model.layers))


flatten_layer = Flatten()
dense_layer1 = Dense(256, activation='relu')
dense_layer2 = Dense(128, activation='relu')
dense_layer3 = Dense(62, activation='relu')
dense_layer4 = Dense(1, activation=tf.nn.sigmoid)

model = Sequential([
        base_model,
        flatten_layer,
        dense_layer1,
        dense_layer2,        
        dense_layer3,
		dense_layer4,		
        ])

base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()
 
model.fit(train_img, train_label, epochs=10, batch_size=16, validation_data = (test_img, test_label))

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

# 만들어 진 모델 저장하기

In [8]:
# save model
model.save("mask_detector.model.h5")
 
print("Saved model to disk") 

Saved model to disk
